In [1]:
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pixel as pix
import utilities as util
import tile
import importlib
import resnet as rs
import keras
from keras.optimizers import Adam
importlib.reload(tile)

Using TensorFlow backend.


<module 'tile' from '/host/Desktop/cnn_dev/florence_mapping/tile.py'>

In [2]:
landsat, sentinel, dem, label = util.load_data()
tile_size = 64
num_classes = util.get_class_count()
pixel_gen = pix.pixel_gen(landsat, sentinel, dem, label, tile_size, num_classes)
tile_gen = tile.tile_gen(landsat, sentinel, dem, label, tile_size, num_classes)

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="3";

In [ ]:
#pixels = pixel_gen.gen_pixels(1000000, not_include=5, balanced=True)
#train_px, val_px, test_px = pixel_gen.train_val_test_split(pixels, 0.7, 0.7)
#pixel_gen.print_balance()

In [4]:
pixels = util.read_txt(["train_px.txt", "val_px.txt", "test_px.txt"])

In [5]:
train_px, val_px, test_px = (pixels[0], pixels[1], pixels[2])

In [6]:
epochs = 20
batch_size = 25
input_shape = tile_gen.get_tile_shape(reshape=True)
print(input_shape)

(10, 64, 64)


In [7]:
model = rs.ResnetBuilder.build_resnet_101(input_shape,num_classes)

In [8]:
opt = Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit_generator(generator=tile_gen.tile_generator(train_px, batch_size), 
                    steps_per_epoch=len(train_px) // batch_size, epochs=epochs, verbose=1,
                    validation_data=tile_gen.tile_generator(val_px, batch_size),
                    validation_steps=len(val_px) // batch_size)

Epoch 1/20
20791/20791 [==============================] - 15760s 758ms/step - loss: 1.5104 - acc: 0.5286 - val_loss: 1.6191 - val_acc: 0.5019
Epoch 2/20
20791/20791 [==============================] - 13157s 633ms/step - loss: 1.5024 - acc: 0.5313 - val_loss: 1.5708 - val_acc: 0.5094
Epoch 3/20
20791/20791 [==============================] - 12682s 610ms/step - loss: 1.4958 - acc: 0.5330 - val_loss: 1.5471 - val_acc: 0.5208
Epoch 4/20
20791/20791 [==============================] - 12988s 625ms/step - loss: 1.4896 - acc: 0.5357 - val_loss: 1.5676 - val_acc: 0.5054
Epoch 5/20
20791/20791 [==============================] - 12531s 603ms/step - loss: 1.4857 - acc: 0.5368 - val_loss: 1.5954 - val_acc: 0.5065
Epoch 6/20
20791/20791 [==============================] - 12559s 604ms/step - loss: 1.4812 - acc: 0.5377 - val_loss: 1.5336 - val_acc: 0.5188
Epoch 7/20
20791/20791 [==============================] - 12256s 590ms/step - loss: 1.4780 - acc: 0.5382 - val_loss: 1.5886 - val_acc: 0.5112
Epoch 

In [12]:
model.save('resnet_fixed.h5')

# Test Pixels

In [ ]:
tile_gen.evaluate_cnn(model, test_px)

# Test Pixels on specific tiles

In [ ]:
pixel_buckets = { 0: list(), 1: list(), 2: list(), 3: list(), 4: list()}
for pix in test_px:
    pixel_buckets[pix[1]].append(pix)

# Tile 0

In [ ]:
tile_gen.evaluate_cnn(model, pixel_buckets[0])

# Tile 1

In [ ]:
tile_gen.evaluate_cnn(model, pixel_buckets[1])

# Tile 2

In [ ]:
tile_gen.evaluate_cnn(model, pixel_buckets[2])

# Tile 3

In [ ]:
tile_gen.evaluate_cnn(model, pixel_buckets[3])

# Tile 4

In [ ]:
tile_gen.evaluate_cnn(model, pixel_buckets[4])